In [1]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import DictionaryLearning, sparse_encode

# Load data
measurement = np.load('../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

In [7]:
# Training data
alice_train_CIRs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
eve_train_CIRs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
alice_train_magnitude = np.abs(alice_train_CIRs[..., 0] + 1j * alice_train_CIRs[..., 1])  # (6400, 251)
eve_train_magnitude = np.abs(eve_train_CIRs[..., 0] + 1j * eve_train_CIRs[..., 1])        # (6400, 251)

# Combine training CIRs
train_cirs = np.vstack((alice_train_magnitude, eve_train_magnitude))

# Labels
alice_train_labels = np.zeros(alice_train_CIRs.shape[0])  # Label '0' for Alice.
eve_train_labels = np.ones(eve_train_CIRs.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((alice_train_labels, eve_train_labels))


In [ ]:
# Reshape data: Flatten the real and imaginary parts into a single vector
# def preprocess_data(data):
#     reshaped_data = data.reshape(data.shape[0], -1)  # Shape: (num_samples, 251*2)
#     scaler = StandardScaler()
#     data_scaled = scaler.fit_transform(reshaped_data)
#     return data_scaled, scaler

# train_data_scaled, scaler = preprocess_data(train_cirs)
# print('Training data shape:', train_data_scaled.shape)


Training data shape: (1600, 251)


In [ ]:
# Learn an overcomplete dictionary from the training data
n_components = 4  # Number of atoms in the dictionary; can be adjusted
dict_learner = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars')
D = dict_learner.fit(train_data_scaled).components_  # Shape: (n_components, n_features)
print('Dictionary shape:', D.shape)

Dictionary shape: (4, 251)


In [ ]:
# Get sparse codes for training data
# train_codes = dict_learner.transform(train_data_scaled)  # Shape: (num_samples, n_components)
# print('Sparse codes shape:', train_codes.shape)

Sparse codes shape: (1600, 4)


In [28]:
# Testing data
alice_test_CIRs = testCIR[:, alice_channel, :, :]
eve_test_CIRs = testCIR[:, eve_channel, :, :]
alice_test_magnitude = np.abs(alice_test_CIRs[..., 0] + 1j * alice_test_CIRs[..., 1])
eve_test_magnitude = np.abs(eve_test_CIRs[..., 0] + 1j * eve_test_CIRs[..., 1])
test_cirs = np.vstack((alice_test_magnitude, eve_test_magnitude))


# Labels
alice_test_labels = np.zeros(alice_test_CIRs.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(eve_test_CIRs.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((alice_test_labels, eve_test_labels))

# Preprocess testing data
reshaped_test_cirs = test_cirs.reshape(test_cirs.shape[0], -1)
test_data_scaled = scaler.transform(reshaped_test_cirs)


In [ ]:
# Get sparse codes for testing data using the same dictionary
# test_codes = sparse_encode(test_data_scaled, D, algorithm='lasso_lars', n_nonzero_coefs=4)
# print('Test codes shape:', test_codes.shape)


Test codes shape: (400, 4)


In [ ]:
# Coefficients calculation
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=15):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_


def calculate_residual(tSample, coefficients, selected_atoms, D):
    # Set the coefficients that are not in the selected atoms to zero
    coef_class = np.zeros_like(coefficients)
    coef_class[selected_atoms] = coefficients[selected_atoms]
    # Reconstruct the signal using the selected atoms and their coefficients
    # reconstructed_signal = np.dot(coef_class, D)
    # Calculate the residual (difference between original and reconstructed signal)
    # residual = np.linalg.norm(tSample - reconstructed_signal)
    # return residual

def classify_signal(tSample, D, train_labels, n_nonzero_coefs=2):
    # Compute sparse code for the test sample
    coefficients = coefficients = find_sparse_coefficients(tSample, D)
    print('Coefficients:', coefficients.shape)
    residuals = []
    unique_classes = np.unique(train_labels)
    for class_label in unique_classes:
        class_indices = np.where(train_labels == class_label)[0]
        residual = calculate_residual(tSample, coefficients, class_indices, D)
    #     # residuals.append(residual)
    # predicted_class = unique_classes[np.argmin(residuals)]
    # return predicted_class


In [45]:
predictions = []

for i in range(test_data_scaled.shape[0])[:2]:
    test_sample = test_data_scaled[i]
    predicted_class = classify_signal(test_sample, D, train_labels, n_nonzero_coefs=2)
    predictions.append(predicted_class)

predictions = np.array(predictions)


ValueError: Expected 2D array, got 1D array instead:
array=[ 7.82597590e-04 -7.91539107e-01  5.63207734e-01  1.05022745e+00
  7.35486527e-01 -1.71309747e-01 -1.65274951e+00 -3.11959464e-01
  2.58245359e-01  7.82428926e-01  4.89461641e-01 -1.18641614e+00
 -3.16238265e-01  5.88922785e-01  4.46219106e-01  5.77216021e-01
  1.18752432e+00  1.94493182e+00  4.03487539e-01  5.60754626e-01
  1.38109039e+00  2.40340488e-01 -8.86979533e-01  1.22451668e-01
  5.65059746e-01 -1.25829779e-01 -1.24990639e+00 -9.74690371e-01
 -3.08888984e-02 -1.23754965e+00 -1.08135481e+00 -1.04245357e+00
 -7.93843204e-01 -8.03016071e-01 -1.63113205e-01  4.74700822e-01
  1.74905040e+00 -3.97020429e-01 -1.03777028e+00  1.51657553e-01
  5.44975287e-01 -2.61934016e-01  5.22807144e-02 -4.38809042e-01
 -5.50156479e-01  1.11837353e-01  2.61979763e-01 -1.59794011e-01
 -3.15021898e-01 -1.86526248e-01 -9.22069066e-03  3.62797299e-01
  4.03759343e-01  6.23359853e-01  1.62983729e-01 -8.69237279e-01
  3.65243670e-01 -1.50556620e+0

In [ ]:
accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"True Positives (tp): {tp}")
print(f"True Negatives (tn): {tn}")
print(f"False Positives (fp): {fp}")
print(f"False Negatives (fn): {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0

# False Alarm Rate (FAR)
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0

# Gamma calculation
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0

# Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


In [2]:
import numpy as np
from sklearn.decomposition import DictionaryLearning

# Step 1: Create a dummy dataset (100 samples, 251 data points, 2 features)
data = np.random.randn(100, 251, 2)  # Shape: (100, 251, 2)

# Step 2: Set up Dictionary Learning
n_components = 5  # The number of components to reduce to
dict_learning = DictionaryLearning(n_components=n_components, transform_algorithm='omp', transform_n_nonzero_coefs=5)

# Step 3: Apply Dictionary Learning to each CIR individually
reduced_data = []

for cir in data:
    # Apply Dictionary Learning to the real and imaginary parts separately
    # Step 3.1: Reshape CIR to be compatible with Dictionary Learning (251, 2)
    # The shape (251, 2) will be treated as the number of samples and features.
    cir_reshaped = cir  # Shape: (251, 2)

    # Step 3.2: Fit and transform using Dictionary Learning
    cir_sparse = dict_learning.fit_transform(cir_reshaped)  # Shape: (251, 5)

    # Step 3.3: Append the transformed CIR
    reduced_data.append(cir_sparse)

# Step 4: Convert the reduced data back to NumPy array
reduced_data = np.array(reduced_data)  # Shape: (100, 5, 2)

print(reduced_data.shape)  # Should output: (100, 5, 2)


(100, 251, 5)


(100, 251, 2)
